## Import of required Modules
Make sure, that the imported moduls are installed.

In [1]:
import pickle
import re
from tqdm import tqdm
import time
import numpy as np
from IPython.display import clear_output 
import pandas as pd
import time

In [2]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

### Settings
change the settings to assert the scraper runs on your system. We reccomend to change the wait_time to a higher value if you run the scraper on a slower system or internet connection.

In [3]:
wait_time = 0.25

#### set up Selenium and Chrome Driver 
We use selenium with Chrome and tested the scraper with the chromedriver. You need the latest version of the driver from https://chromedriver.chromium.org/. Alternatively, change to the driver to a driver of your preferance.
We set up the scraper to run in the background, if you wish to run it in regular window mode, remove the line  "chrome_options.add_argument("--headless")".

In [ ]:
# chromedriver setup

# serv = Service(r'driver/chromedriver') #path from 'which chromedriver'

# test driver
# for headless chrome mode
# chrome_options = Options()

# remove this line if you do not wish to run in background 
# chrome_options.add_argument("--headless") 

## Retrieve news source bias information

In [23]:


chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options = chrome_options)


# driver.get('https://www.allsides.com/media-bias/ratings')

file_path = "/media/himel/Data/Documents/Academic/Phd_Prep/Qbias/Scraper/Media Bias Ratings _ AllSides.html"
driver.get("file://" + file_path)

main_table = driver.find_element(By.XPATH, "//*[@id=\"content\"]/div/div[2]/table[1]/tbody") # table body

rows = main_table.find_elements(By.TAG_NAME, "tr")
columns = ["News Source", "Link", "AllSides Bias Rating", "Community feedback" ]

data = []
for i in rows:
    entry = i.find_elements(By.TAG_NAME, "td")
    news_source = entry[0].find_element(By.TAG_NAME, "a").text
    link = entry[0].find_element(By.TAG_NAME, "a").get_attribute("href")
    bias_rating = entry[1].find_element(By.TAG_NAME, "a").get_attribute("href").split('/')[-1]
    communtiy_feedback = entry[3].find_element(By.CLASS_NAME, "commtext").text
    data.append([news_source, link, bias_rating, communtiy_feedback])
driver.quit()
df  = pd.DataFrame(data, columns=columns)

df

,News Source,Link,AllSides Bias Rating,Community feedback
0,ABC News (Online),https://www.allsides.com/news-source/abc-news-...,left-center,Community strongly agrees.
1,AlterNet,https://www.allsides.com/news-source/alternet-...,left,Community absolutely agrees.
2,Associated Press,https://www.allsides.com/news-source/associate...,left,Community somewhat agrees.
3,Associated Press Fact Check,https://www.allsides.com/news-source/ap-fact-c...,left-center,Community somewhat disagrees.
4,Axios,https://www.allsides.com/news-source/axios,left-center,Community somewhat disagrees.
5,BBC News,https://www.allsides.com/news-source/bbc-news-...,center,Community somewhat agrees.
6,Bloomberg,https://www.allsides.com/news-source/bloomberg...,left-center,Community somewhat disagrees.
7,Breitbart News,https://www.allsides.com/news-source/breitbart,right,Community absolutely agrees.
8,Business Insider,https://www.allsides.com/news-source/insider-m...,left-center,Community somewhat disagrees.
9,BuzzFeed News,https://www.allsides.com/news-source/buzzfeed-...,left,Community strongly agrees.


In [24]:
df.to_csv("allsides_news_source_bias_ratings.csv", index=False)

### Retrieve Article Links for Crawl

In [8]:
# assemble list of links to all articles
links = []

# interact with cookie terms
# chrome_path = r'driver/chromedriver' #path from 'which chromedriver'
# chrome_options = Options()
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--headless") # open chrome in background
# driver = webdriver.Chrome(service=serv, options=chrome_options)
driver = webdriver.Chrome(options = chrome_options)

driver.get('https://www.allsides.com/headline-roundups')
wait = WebDriverWait(driver, 10)
# wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'css-47sehv')))

# ele = driver.find_element(By.CLASS_NAME, "css-47sehv")
# ele.click()

# retireve number of pages 
last_page_button = driver.find_element(By.CLASS_NAME, "pager-last")
link_last_page = last_page_button.find_elements(By.TAG_NAME, "a")
t = link_last_page[0].get_attribute("href")
last_page_index = int(t[-3:])

# retrieve links from start page
main_table = driver.find_element(By.XPATH, "//*[@id=\"block-views-de37fa32ea86f5545eb9b7722977a70d\"]/div/div[2]/table/tbody") # table body

rows = main_table.find_elements(By.TAG_NAME, "tr")
for i in rows:
    entry = i.find_elements(By.TAG_NAME, "td")
    link = entry[0].find_element(By.TAG_NAME, "a")
    links.append(link.get_attribute("href"))
WebDriverWait(driver, 20)

# retrieve links for other pages
for page in tqdm(range(2,last_page_index+1)):   # set to max number of pages
    driver.get("https://www.allsides.com/headline-roundups?page="+str(page))
    WebDriverWait(driver, 20)
    main_table = driver.find_element(By.XPATH, "//*[@id=\"block-views-de37fa32ea86f5545eb9b7722977a70d\"]/div/div[2]/table/tbody") # table body

    rows = main_table.find_elements(By.TAG_NAME, "tr")
    for i in rows:
        entry = i.find_elements(By.TAG_NAME, "td")
        link = entry[0].find_element(By.TAG_NAME, "a")
        links.append(link.get_attribute("href"))

driver.close()

ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [74]:
len(links)

10547

In [75]:
# exports links as pickle file
with open("linklist_allsides_news.pickle", "wb") as f:
    pickle.dump(links, f, pickle.HIGHEST_PROTOCOL)

In [76]:
# exports links as csv
with open("linklist_allsides_news.csv", "w") as f:
    for line in links:
        print(line, file=f)

## Retrieve Articles
This section of the crawler retrieves all available news articles from AllSides along with the available information and bias tags.

In [41]:
# functions that assert the existence of 
def check_exists_by_xpath(xpath):
    try:
        driver.find_elements(By.XPATH, xpath)[0]
    except:
        return False
    return True

def check_exists_by_class(inp):
    try:
        driver.find_elements(By.CLASS_NAME, inp)[0]
    except:
        return False
    return True

In [58]:
# load link list from pickle file
with open("linklist_allsides_news.pickle", "rb") as f:
    links = pickle.load(f)

len(links)

10547

In [59]:
batch_size = 1000
n_done = 2
start = 0 + batch_size*n_done
end = start+batch_size
links = links[start:end]
print(links[0])
print(links[-1])
len(links)

https://www.allsides.com/story/politics-foreign-aid-package-divides-house-senate-republicans
https://www.allsides.com/story/abortion-fda-expands-abortion-pill-access


1000

In [63]:
# list for results 
data = []
wait_time = 0.5
WINDOW_SIZE = "1920,1080"
CHROME_PATH = '/usr/bin/google-chrome'
CHROMEDRIVER_PATH = '/media/himel/Data/Documents/Academic/Phd_Prep/chromedriver-linux64/chromedriver'
# retrieve information from articles in list of links
for li in tqdm(links):
    time.sleep(wait_time)
    # driver = webdriver.Chrome(service=serv, options=chrome_options)
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    # value got from: https://useragentstring.com/
    chrome_options.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36")
    # chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
    # chrome_options.binary_location = CHROME_PATH
    driver = webdriver.Chrome(
        # executable_path=CHROMEDRIVER_PATH, 
        options = chrome_options
                              )

    print(li)

    # open URL
    driver.get(li)

    # test headless ui
    # driver.get_screenshot_as_file("capture.png")
    # driver.close()
    # break
    
    # interact with pop-up window
    if check_exists_by_class("css-47sehv"):
        ele = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((driver.find_element(By.CLASS_NAME, "css-47sehv"))))
        ele.click()
    else:
        print("no button")
    
    # netral title heading 
    try:
        heading = driver.find_element(By.TAG_NAME, "h1").text
        description = '\n'.join([p.text for p in driver.find_element(By.CLASS_NAME, "story-id-page-description").find_elements(By.TAG_NAME, "p")])
    except:
        print("no heading found")
    
    print(heading)
    
    # date
    try:
        date = driver.find_element(By.CLASS_NAME, "date-display-single").text
    except:
        date = ""
    
    # tags
    try:
        tags = [a.text for a in driver.find_element(By.CLASS_NAME, "page-tags").find_elements(By.TAG_NAME, "a")]
    except:
        tags = ""
        
    
    # define XPATH inforamtion for article divs
    # divs = ["/html/body/div[4]/div/div/div/div[4]/div/div/div/div[1]", "/html/body/div[4]/div/div/div/div[4]/div/div/div/div[2]", "/html/body/div[4]/div/div/div/div[4]/div/div/div/div[3]"]
    divs = ["/html/body/div[2]/div/div/div[2]/div[4]/div/div/div/div[1]", 
            "/html/body/div[2]/div/div/div[2]/div[4]/div/div/div/div[2]", 
            "/html/body/div[2]/div/div/div[2]/div[4]/div/div/div/div[3]"]
   


    # access information in article divs
    for d in divs:
        if check_exists_by_xpath(d):
            div = driver.find_elements(By.XPATH, d)[0]

            # check heading element to find out left/center/right. The title contains the bias label that we can retrieve from the text here
            try:
                cat = div.find_element(By.TAG_NAME, "h3").text
            except: 
                print("no headline found")
                
            # retrieve link to original article
            try:
                link = div.find_element(By.TAG_NAME, "a").get_attribute("href")
                print(link)
            except:
                print("no link found")
                
            # left/center/right are shuffled for each article, some roundups have e.g. only left and right articles. 
            # Thus, we have look to look each article seperately
            time.sleep(0.2)
            if "Left" in cat:
                print("left")
                try:
                    left_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading       
                except:
                    left_heading = ""
                    print("no headline found")
                try:
                    left_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source
                except:
                    left_source = ""
                    print("no source found")
                try:
                    left_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text body-contents
                except:
                    left_text = ""
                    print("no text found")
                    
                # add the article information
                data.append({"url":link, "date":date, "title": heading, "description": description, "tags": tags, 
                             "heading":left_heading, "source": left_source, "text": left_text, "bias_rating": "left"})

            elif "Right" in cat:
                print("right")
                try:
                    right_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading
                except:
                    right_heading = ""
                    print("no headline found")
                try:
                    right_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source 
                except:
                    right_source = ""
                    print("no source found")
                try:
                    right_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text
                except:
                    right_text = ""
                    print("no text found")
                
                # add the article information
                data.append({"url":link, "date":date, "title": heading, "description": description, "tags": tags, 
                             "heading":right_heading, "source": right_source, "text": right_text, "bias_rating": "right"}) 

            else:
                print("center")
                try:
                    center_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading
                except:
                    center_heading = ""
                    print("no headline found")
                try:
                    center_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source 
                except:
                    center_source = ""
                    print("no source found")
                try:
                    center_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text
                except:
                    center_text = ""
                    print("no text found")
                    
                # add the article information
                data.append({"url":link, "date":date, "title": heading, "description": description, "tags": tags, 
                             "heading":center_heading, "source": center_source, "text": center_text, "bias_rating": "center"})
        else:
            print("div not found")
        
    
    driver.close()
    
    # clear output
    clear_output()
    # added a wait here to assert the scraper runs well
    time.sleep(wait_time)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:52:41<00:00,  6.76s/it]


In [62]:
len(data)

393

In [55]:
import os

if os.path.exists("allsides_news_complete.csv"):
    old_df = pd.read_csv("allsides_news_complete.csv", index_col=0)
    df = pd.concat([old_df, pd.DataFrame(data)]).reset_index(drop=True)
else:
    ## convert data to dataframe
    df = pd.DataFrame(data)

# export scraped articles
df.to_csv("allsides_news_complete.csv")

In [57]:
df = pd.read_csv("allsides_news_complete.csv", index_col=0)
print(df.title.unique().shape)
print(df.bias_rating.value_counts())
df

(1771,)
bias_rating
left      1864
right     1774
center    1576
Name: count, dtype: int64


,url,date,title,description,tags,heading,source,text,bias_rating
0,https://edition.cnn.com/2025/04/11/politics/us...,"April 13th, 2025",Trump Orders Military to Control Land on South...,President Donald Trump ordered the US military...,"['Defense And Security', 'US-Mexico Border', '...",Trump authorizes military to take control of f...,CNN Digital,President Donald Trump sent a memorandum to fo...,left
1,https://www.newsnationnow.com/us-news/immigrat...,"April 13th, 2025",Trump Orders Military to Control Land on South...,President Donald Trump ordered the US military...,"['Defense And Security', 'US-Mexico Border', '...",White House authorizes military control of lan...,NewsNation,The newest effort from the Trump administratio...,center
2,https://www.foxnews.com/politics/trump-orders-...,"April 13th, 2025",Trump Orders Military to Control Land on South...,President Donald Trump ordered the US military...,"['Defense And Security', 'US-Mexico Border', '...",Trump orders military to take control of feder...,Fox News Digital,The U.S. military will take control of a strip...,right
3,https://nypost.com/2025/04/12/world-news/iran-...,"April 12th, 2025",Iran and US Conduct Preliminary Nuclear Talks ...,Envoys from Iran and the US met in Oman to com...,"['Foreign Policy', 'Donald Trump', 'Iran', 'Nu...",Iran says ‘indirect talks’ begin with US envoy...,New York Post (News),Iran and the United States will hold more nego...,right
4,https://www.bbc.com/news/articles/c4g2eggzvjgo,"April 12th, 2025",Iran and US Conduct Preliminary Nuclear Talks ...,Envoys from Iran and the US met in Oman to com...,"['Foreign Policy', 'Donald Trump', 'Iran', 'Nu...",Iran says it wants 'fair agreement' as nuclear...,BBC News,Iran and the United States have begun talks in...,center
...,...,...,...,...,...,...,...,...,...
5209,https://www.washingtonpost.com/politics/2023/1...,"November 2nd, 2023",Can We Trust Gaza Death Toll Numbers?,Gaza's Hamas-run health ministry says nearly 9...,"['Middle East', 'Israel Hamas Violence', 'Gaza...",Biden’s dismissal of the reported Palestinian ...,Washington Post Fact Check,The Hamas attack on Israel on Oct. 7 killed mo...,left
5210,https://www.wsj.com/livecoverage/israel-hamas-...,"November 2nd, 2023",Can We Trust Gaza Death Toll Numbers?,Gaza's Hamas-run health ministry says nearly 9...,"['Middle East', 'Israel Hamas Violence', 'Gaza...","Gaza Death Toll Rises to More Than 9,000, Loca...",Wall Street Journal (News),The Hamas-controlled Gaza health authorities s...,center
5211,https://www.thedailybeast.com/elon-musk-spins-...,"November 1st, 2023",Musk Says Twitter Suppressed Republicans at '1...,Tesla CEO and X (formerly Twitter) owner Elon ...,"['Media Bias', 'Social Media', 'Twitter', 'X',...","Elon Musk Goes Buck Wild in Really, Really Lon...",Daily Beast,Elon Musk made his fourth appearance Tuesday o...,left
5212,https://www.newsweek.com/musk-tells-rogan-twit...,"November 1st, 2023",Musk Says Twitter Suppressed Republicans at '1...,Tesla CEO and X (formerly Twitter) owner Elon ...,"['Media Bias', 'Social Media', 'Twitter', 'X',...",Musk Tells Rogan Twitter 'Suppressed' Republic...,Newsweek,"Elon Musk said that conservative users of X, f...",center
